In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [88]:
def clear_gpu():
    import gc, torch
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_gpu()


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

# Classification

In [4]:
import json
file_path = "/kaggle/input/classification/classification_instruction_data_en.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f]

In [5]:
valid_labels = {"negative", "positive", "neutral"}
data = [sample for sample in data if sample['output'].strip().lower() in valid_labels]

In [6]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]  
    return None

In [7]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [8]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

**Zero-shot**

In [9]:
acc = 0
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: {data[i]['input'] } A: The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
positive
positive
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
negative
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
The answer is ?
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutr

Zero-shot + CoT (ReAct)

In [13]:
acc = 0
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}.
  Q: {data[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
print('Accuracy', acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
negative
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral

In [11]:
acc = 0
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: {data[i]['input'] } A: Let's think step by step. The sentiment is """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

0.8


Zero-shot + ToT

In [84]:
def zero_shot_ToT(text):
    return (
        f"{text['instruction']}. Q: {text['input']}.\n"
        f"A: Let's think of three strategies for this question.\n"
        f"Strategy 1: "
    )

In [43]:
import re

def extract_labels_from_thoughts(text):
    matches = re.findall(r'\b(positive|negative|neutral)\b', text.lower())
    return list(set(matches))  # loại trùng nếu cần


In [80]:
import re

def clean_label(text):
    match = re.search(r"\b(positive|negative|neutral)\b", text.lower())
    return match.group(1) if match else "UNKNOWN"


In [41]:
def generate_strategy(prompt_base, strategy_index):
    prompt = prompt_base + f"Strategy {strategy_index + 1}:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [207]:
acc = 0
for i in range(200):
    prompt_base = zero_shot_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc+1
print(acc/200)

0.66


In [206]:
acc = 0
for i in range(200):
    prompt_base = zero_shot_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(5)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc+1
print(acc/200)

0.6


In [208]:
acc = 0
for i in range(200):
    prompt_base = zero_shot_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(10)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc+1
print(acc/200)

0.6


Zero-shot + CoT + SC

In [59]:
from collections import Counter
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=50,
            do_sample=True,           
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    counts = Counter(outputs)
    return counts.most_common(1)[0][0], counts

In [92]:
def zero_shot_CoT(text):
  return f"""
  You are a classifier assistant.{text['instruction']}. 
  Q: {text['input'] } A: Let's think step by step. The sentiment is """

In [99]:
prompt = zero_shot_CoT(data[0])
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)


  You are a classifier assistant.Classify the following text into 'positive', 'neutral', or 'negative'.. 
  Q: between 1900 and 1920 where did most of the migrants to the united states come from A: Let's think step by step. The sentiment is 
Self-Consistent Answer: neutral
All votes: Counter({'neutral': 5})


In [101]:
acc = 0
for i in range(200):
    prompt = zero_shot_CoT(data[i])
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    if best_answer == data[i]['output']:
        acc = acc + 1
    print(best_answer)
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negative
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negat

In [142]:
acc = 0
for i in range(200):
    prompt = zero_shot_CoT(data[i])
    samples = self_consistency(prompt, num_samples=10)
    best_answer, all_votes = majority_vote(samples)
    if best_answer == data[i]['output']:
        acc = acc + 1
print(acc/200)

0.78


**Few-shots**

Few-shot + CoT (ReAct)

In [103]:
acc = 0
results = []
for i in range(200):
  few_shots_prompt = f"""
  You're a sentiment classifier assistant. {data[i]['instruction']}, Recorrect each time you classifying with other LLM model
  Example:Although I had high hopes, this product was a huge disappointment.
  Let's think step by step.
  1. The person had high hopes.
  2. But the product was a disappointment.
  3. The overall sentiment is negative.
  Q: {data[i]['input'] } A: Let's think step by step. The sentiment is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

0.815


In [104]:
acc = 0
results = []
for i in range(200):
  few_shots_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: hi, A: positive
  Q: Efficient and fast, A: positive
  Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?, A: neutral
  Q: {data[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
neutral
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral

*thử đảo vị trí các ví dụ*

In [107]:
acc = 0
results = []
for i in range(3, 200):
  few_shots_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?, A: neutral
  Q: I hate this chat, A: negative
  Q: Efficient and fast, A: positive
  Q: {data[i]['input'] } A: The sentiment is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
print(acc/197)

0.8071065989847716


Few-shots + CoT + SC

In [ ]:
def few_shots_CoT_SC(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)
for i in range(50):
    few_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Few-shots + Self-evaluation ToT 

In [195]:
def few_shots_ToT(text):
    return f"""
  You are a classifier assistant.{text['instruction']}. Recorrect each time you classifying with other LLM model
  Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?, A: neutral
  Q: I hate this chat, A: negative
  Q: Efficient and fast, A: positive
  Q: {text['input'] } A: The sentiment is ?"""

In [203]:
acc = 0
for i in range(200):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc + 1
print(acc/200)

0.82


In [201]:
acc = 0
for i in range(200):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(5)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc + 1
print(acc/200)

0.71


In [202]:
acc = 0
for i in range(200):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(10)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc + 1
print(acc/200)

0.635


# QA

(hỏi đáp đơn giản)

In [213]:
clear_gpu()

**Zero-shot**

In [3]:
import json
qa = []
with open('/kaggle/input/simple-qa/simple_qa.json', 'r', encoding='utf-8') as f:
    qa.append(json.load(f))

In [4]:
qa = qa[0]

In [223]:
qa[:3]

[{'instruction': 'Answer the question',
  'input': 'between 1900 and 1920 where did most of the migrants to the united states come from',
  'output': 'During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.'},
 {'instruction': 'Answer the question',
  'input': 'What does inioluwa mean',
  'output': 'Inioluwa is a Nigerian name that means "God\'s gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.'},
 {'instruction': 'Answer the question',
  'input': "Today let's pretend we are building a 16-bit computer.",
  'output': "Sure, I'd be happy to help you with building a 16-bit computer. What specific components or aspects of the computer would you lik

In [225]:
def zero_shot_prompt(text):
    return f"Q: {text['input']}. A: Answer the question "

In [229]:
print(qa[0]['input'])
print(qa[0]['output'])

between 1900 and 1920 where did most of the migrants to the united states come from
During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.


In [227]:
input = tokenizer(zero_shot_prompt(qa[0]), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Mexico


In [247]:
test_error = """Q: In the early 20th century, where did most of the migrants to the united states come from
. A: Answer the question, which's country ? """

input = tokenizer(test_error, return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


finland


moi lan mot nuoc ~

In [251]:
print(qa[1]['input'])
print(qa[1]['output'])

What does inioluwa mean
Inioluwa is a Nigerian name that means "God's gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.


In [256]:
input = tokenizer(zero_shot_prompt(qa[1]), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

(IV).


Zero-shot + CoT

In [22]:
def zero_shot_CoT(text):
    return f"Q: {text['input']}. A: Let's think step by step and explain why"

In [35]:
clear_gpu()

In [32]:
clear_gpu()

In [6]:
input = tokenizer(zero_shot_CoT(qa[0]), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The relevant information to answer the above question is: Between 1900 and 1920, most of the migrants to the United States came from Europe. So the final answer is Europe.


EM

In [11]:
import re

def extract_final_answer(text):
    match = re.search(r'final answer is\s*(.+?)(?:\.|$)', text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    sentences = re.split(r'[.?!]', text)
    if sentences:
        return sentences[-2].strip()
    return text.strip()


In [12]:
def exact_match(prediction, ground_truth):
    pred_ans = extract_final_answer(prediction)
    gt_ans = extract_final_answer(ground_truth)
    
    def normalize_text(text):
        text = text.lower().strip()
        text = re.sub(r'[^\w\s]', '', text)
        return text
    
    return normalize_text(pred_ans) == normalize_text(gt_ans)

In [23]:
for i in range(10):
    input = tokenizer(zero_shot_CoT(qa[i]), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)

The relevant information to answer the above question is: Between 1900 and 1920, most of the migrants to the United States came from Europe. So the final answer is Europe.
To be kind, generous, or to be generous means to be kind and generous to others. So the answer is to bekind and generous.
The 16-bit computer was a great leap forward from the 8-bit and 32-bit computers of the 1980s and 1990s.
Property listing, real estate marketing or MLS (multiple listing service) is a way of putting properties on the real estate market.
Tom is the one who threw the school bag down to Ray. The final answer: Tom.
hi.
5.
The dog lost $12 - $12 = $1. So the answer is 1.
The relevant information to answer the above question is: Write a song about it in the style of Blackpink then. So the answer is write a track about it.
The 1st ball is lighter, so it is heaviest. The second ball is also lighter, but is a heavier ball. This is because, if you have 2 identical balls, one of them is lighter than the othe

In [56]:
from sentence_transformers import SentenceTransformer, util
comp_model = SentenceTransformer('all-MiniLM-L6-V2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [57]:
def cosine_similarity_score(pred, target):
    emb1 = comp_model.encode(pred, convert_to_tensor = True)
    emb2 = comp_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])

In [10]:
print('Target', qa[9]['output'])
print('Prediction', answer)

Target That's correct! With 3 balls, we can easily find the heaviest one in just one comparison. However, with 8 balls and only two attempts allowed, we need to come up with a more optimal strategy. Keep trying, you're on the right track!
Prediction The difference in weight of the balls is equal to their difference in mass. So if a ball is lighter than a heaviest ball, it is lighter. However if the balls are the same weight, the heavest ball is the smallest, if not the lightest ball is heavest. So the answer is yes.


In [11]:
cosine_similarity_score(answer, qa[9]['output'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.5950660705566406

In [24]:
cs = []
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cs.append(c)

The most migrants to the United States between 1900 and 1920 were from Asia. The most migrant groups were from Europe, the Middle East, and Africa. So the final answer is Europe.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The term inioluwa , meaning "to bring to light" or "to show forth" in the NIgalese language, derives from the wà , a verb meaning "let shine." So the answer is to bring to Light.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

I want you to act as a coach for the interview.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
tem = 0
for i in range(len(cs)):
    tem = tem +cs[i]
print(tem/len(cs))

0.42192325871437786


In [31]:
pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8b5415cb9bcfe89a05bc8469ff2a0ec920672725e1537250abcdebdea633441e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [35]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
score = scorer.score(answer,qa[49]['output'])
print(score)

{'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


In [36]:
def compute_f1(pred, target):
    pred_tokens = set(pred.lower().split())
    target_tokens = set(target.lower().split())
    common = pred_tokens & target_tokens
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(target_tokens)
    return 2 * precision * recall / (precision + recall)
print(compute_f1(answer, qa[49]['output']))

0.0


In [55]:
def zero_shot_CoT(text):
    return f"""
    Q: {text['input']}. A: Let's think step by step and explain this question clearly, (include definition if it has).
    You can recorrect with other reliable sources such as wikipedia if you need,.."""

In [56]:
cs = []
for i in range(50):
    input = tokenizer(zero_shot_CoT(qa[i]), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cs.append(c)
    print(c)

The United States Census Bureau reports that in the decade between 1900 and 1920, the United States saw a net inflow of immigrants, while its total number of migrants remained steady. Immigration from Europe was the largest single group of migrants, followed by China and Italy. So, the final answer is Europe.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8093522787094116
The relevant information to answer the above question is: Inioluwa is a village in the Central Province of Cameroon. Iniowa is the only village in Cameron with a population of more than 20,000. So, the answer is iniowe.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.7989904880523682
I want you to act as a coach for the interview. Then you will act as the coach.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.4150424003601074


In [57]:
tem = 0
for i in range(len(cs)):
    tem = tem +cs[i]
print(tem/len(cs))

0.40462451228871943


Zero-shot + CoT + SC

In [62]:
cosine = 0
def zero_shot_CoT(text):
    return f"Q: {text['input']}. A: Let's think step by step and explain why"
for i in range(50):
    prompt = zero_shot_CoT(qa[i])
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    c = cosine_similarity_score(best_answer, qa[i]['output'])
    print(c)
    cosine = cosine+c

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8902917504310608


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8550751209259033


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-0.03663022816181183


In [63]:
print(cosine/50)

0.41921495452523233


**Few-shots**

Few-shots + CoT

In [64]:
cosine = 0
def few_shots_CoT(text):
    return f"""
    Q: {qa[0]['input']}. A: {qa[0]['output']} \n
    Q: {qa[1]['input']}. A: {qa[1]['output']} \n
    Q: {qa[2]['input']}. A: {qa[2]['output']} \n
    Q: {qa[3]['input']}. A: {qa[3]['output']} \n
    Q: {text['input']}. A: Let's think step by step and explain why"""

In [65]:
for i in range(3,53):
    input = tokenizer(few_shots_CoT(qa[i]), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cosine = cosine+c
    print(c)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1.0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.6964414119720459


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8552172183990479


In [68]:
print(cosine/50)

0.4731547723896801


Few-shots + CoT + ART

In [82]:
len(qa)

1083

Lấy thư viện task như 1 tập train lớn và test task sẽ là vùng qa còn lại

In [8]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
new_tasks = qa[:1000]
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)

In [9]:
def find_top_k_tasks(query, k=3):
    q = vectorizer.transform([query])
    scores = cosine_similarity(q, X).flatten()
    top_indices = scores.argsort()[::-1][:k]
    return [task_samples[i] for i in top_indices]

In [10]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        prompt += f"""Task: {ex.get('instruction')}
                    Input: {ex['input']}
                    Output:
                    {ex['output']}
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly"
    )
    return prompt

In [86]:
cosine = 0
def few_shots_CoT_ART(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    return prompt
for i in range(1000, 1050):
    input = tokenizer(few_shots_CoT_ART(qa[i]['input']), return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
    max_length = 300,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)
    c = cosine_similarity_score(answer, qa[i]['output'])
    cosine = cosine+c
    print(c)

Hello! How can I help you today?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.942842960357666
The label MX is assigned to mjshin04. So the final answer is m.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9597274661064148
The relevant information to answer the above question is: A201-2007 is a standard form contract published by the American Institute of Architects (AIA) for use in construction projects involving architecture services. The AIA A201 is  a commonly used contract in the construction industry for projects containing architecture services, and includes updates and changes to the previous editions. The 2007 version of the contract is the most recent edition and includes updated and changes.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.966480016708374


In [87]:
print(cosine/50)

0.9045639550685882


Few-shots + ToT

In [36]:
def few_shots_ToT(text):
    return f"""
   Your task: Write 3 possible reasoning paths or situations that might lead to different conclusions and response. Do not answer the question
    {text['input']} 
    """
def followup_prompt(text):
 return f"""Here are 3 different reasoning paths:  {text}.
 You are a reasoning assistant. Your job is to explore different reasoning paths without concluding.
 Now, expand on each path with clear explanation, but do NOT provide any final answer or conclusion."""


In [37]:
for i in range(50):
    prompt_base = few_shots_ToT(qa[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
            **input_ids,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    print(qa[i]['input'])
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print(answer)

between 1900 and 1920 where did most of the migrants to the united states come from
Strategy 3: Those who came to the United States during the Great Migration in the early 20th century were mostly from Europe, the Far East, and Asia.
What does inioluwa mean
Strategy 2: The term inioluwa is a Yoruba word that means "the father of all". Therefore, the answer is the father.
Today let's pretend we are building a 16-bit computer.
Strategy 1: Let's try to figure out what the 16-bit computer was built with.
can you propose a name for real estate market place?
Strategy 3: if you are a real estate agent you will know that the mls is the only place to buy and sell.
Tom threw his school bag down to Ray after he reached the top of the stairs. Who reached the
top of the stairs?
Strategy 1: Tom threw his school bag down to Ray. Ray reached the top of the stairs. Thus, the answer is Tom.
hi
Strategy 3: You might answer the question hi.
5
Strategy 1: The teacher is asking students to write about what 

# Computation

**Zero-shot**

Zero-shot + CoT

In [1]:
import json
math = []
with open('/kaggle/input/final-train-math/final_train_math.json', 'r', encoding='utf-8') as f:
    math.append(json.load(f))

In [2]:
math = math[0]

In [42]:
prompt = f"Q: {math[0]['input'] } A: Let's think step by step."
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=160)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Natalia sold 48 / 2 = 24 clips in May. Natalia sold a total of 48 + 24 = 96 clips in April and May. The final answer: 96.


In [43]:
math[5]['output']

"There are 80/100 * 10 = <<80/100*10=8>>8 more purple flowers than yellow flowers.\nSo in Mark's garden, there are 10 + 8 = <<10+8=18>>18 purple flowers.\nPurple and yellow flowers sum up to 10 + 18 = <<10+18=28>>28 flowers.\nThat means in Mark's garden there are 25/100 * 28 = <<25/100*28=7>>7 green flowers.\nSo in total Mark has 28 + 7 = <<28+7=35>>35 plants in his garden.\n#### 35"

In [44]:
prompt = f"Q: {math[5]['input'] } A: Let's think step by step. The answer is ?"
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
    **inputs,
    max_length=100,
    num_beams=2,
    no_repeat_ngram_size=3,
    early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

There are 10 * .8 = 8 more purple flowers than yellow flowers. So there are 10 + 8 = 16 purple flowers. And there are 16 * 0.2 = 4 green flowers. Therefore, Mark has 10 + 16 + 4 = 44 flowers in his garden. The answer: 44.


In [3]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]  
    return None

In [47]:
answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(answer)

44


In [4]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [67]:
acc = 0
results = []
for i in range(20):
  zero_shot_prompt = f"""
  You are a math assistant. When you need to calculate something, use the CALC() function.
  Q: {math[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


In [68]:
print(acc/20)

0.2


In [69]:
acc = 0
results = []
few_shots_prompt = f"""
 Q: {math[0]['input']}. Let's think step by step',
  """
inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=2,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

answer =tokenizer.decode(outputs[0], skip_special_tokens=True)
results.append(answer)

In [55]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Natalia sold 48 / 2 = 24 clips in May. Natalia therefore sold a total of 48 + 24 = 96 clips in April and May. The answer: 96.


In [56]:
print(answer)
print(extract_target(math[1]['output']))

Natalia sold 48 / 2 = 24 clips in May. Natalia therefore sold a total of 48 + 24 = 96 clips in April and May. The answer: 96.
10


In [73]:
acc = 0
results = []
for i in range(20):
  zero_shot_prompt = f"""
  You are a math assistant. When you need to calculate something, use the CALC() function.
  Q: {math[i]['input'] } A: You need to answer follow this form: The answer: """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=200,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(de)
  answer = extract_answer(de)
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1

48 / 2 = 24 clips were sold in May. Natalia sold a total of 48 + 24 = 96 clips in April and May.
Weng earned $12 x 50 = $700.
Betty needs $100 / 2 = $50 more to buy the wallet. Betty's grandparents gave her twice as much as her parents, so they gave her 2 * $15 = $30. Together, Betty and her grandparents gave Betty $30 + $15 = $45. Betty still needs $50 - $45 = $25 more.
Today, Julie was able to read 12 x 2 = 24 pages. The total number of pages that she has read is 12 + 24 = 36 pages. If she wants to read half of the remaining pages tomorrow, she should read 1 / 2 * 36 = 18 pages.
He writes 2 * 3 = 6 pages a week. So he writes 6 * 2 = 12 pages per week. That means he wrote 12 * 52 = 552 pages in a year.
There are 10 * .8 = 8 more purple flowers than yellow flowers in Mark's garden. So there are 10 + 8 = 16 purple flowers in his garden. And there are 16 * 0.25 = 4 green flowers. So Mark has a total of 10 + 16 + 4 = 44 flowers.
He eats 2 * 16 + 2 * 8 = 32 pieces of pizza.
gummy worms = 2

In [74]:
print(acc/20)

0.0


In [58]:
for i in range(3):
  print(math[i]['input'])
  print(math[i]['output'])

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5


In [72]:
clear_gpu()

NameError: name 'clear_gpu' is not defined

Zero-shot + CoT + SC

In [75]:
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=150,
            do_sample=True,           # sampling ON
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    answers = [extract_answer(out) for out in outputs]
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [76]:
def zero_shot_CoT_SC(text):
    return f"""
    f"Q: {text['input'] } A: Let's think step by step. The answer is ?"
    """

In [77]:
from collections import Counter
prompt = zero_shot_CoT_SC(math[0])
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)

Natalia sold 48 / 2 = 24 clips in May. Therefore, Natalia sold 48 + 24 = 72 clips altogether in April and May. The answer: 72.
Natalia sold 48 / 2 = 28 clips in May. In April and May, Natalia sold 48 + 28 = 84 clips. The final answer: 84.
Natalia sold 48 / 2 = 28 clips in May. Natalia sold 48 + 28 = 80 clips altogether in April and May. The final answer: 80.
Natalia sold 48 / 2 = 24 clips in May. Therefore, Natalia sold 48 + 24 = 96 clips altogether in April and May. The final answer: 96.
Natalia sold 48 / 2 = 28 clips in May. Natalia sold a total of 48 + 28 = 88 clips in April and May. The answer: 88.
Self-Consistent Answer: 72
All votes: Counter({'72': 1, '84': 1, '80': 1, '96': 1, '88': 1})


In [78]:
acc = 0
for i in range(50):
    prompt = zero_shot_CoT_SC(math[i])
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)
    if best_answer == extract_target(math[i]['output']):
        acc = acc+1

Natalia sold 48 / 2 = 28 clips in May. Natalia sold a total of 48 + 28 = 88 clips in April and May. The final answer: 88.
Natalia sold 48 / 2 = 28 clips in May. Natalia sold a total of 48 + 28 = 84 clips in April and May. The final answer: 84.
Natalia sold 48 / 2 = 24 clips in May. Therefore, Natalia sold 48 + 24 = 84 clips altogether in April and May. The final answer: 84.
Natalia sold 48 / 2 = 24 clips in May. So, Natalia sold a total of 48 + 24 = 96 clips in April and May. The final answer: 96.
Natalia sold 48 / 2 = 24 clips in May. Natalia sold 48 + 24 = 96 clips altogether in April and May. The answer: 96.
Self-Consistent Answer: 84
All votes: Counter({'84': 2, '96': 2, '88': 1})
Weng earns $12 / minute * 50 minutes = $60 for babysitting. The final answer: 60.
Weng earns $12 x 50 = $540 for babysitting. The answer: 540.
Weng earned $12 x 50 = $700 for babysitting yesterday. The final answer: 700.
Weng earns $12 x 50 = $540 for babysitting. The answer: 540.
Weng earned $12 x 50 = $

In [79]:
print(acc/50)

0.08


Zero-shot + ToT

In [85]:
def followup_prompt(text):
 return f"""Here are 3 different reasoning paths:  {text}.
 You are a math assistant. Your job is to explore different reasoning paths without concluding.
 Now, choose the best suitable path with clear explanation and then provide the final answer."""
def zero_shot_ToT(text):
    return f"""
    Q: {text}. A: Answer the question with inferencing
    """
for i in range(50):
    prompt_base = zero_shot_ToT(math[i]['input'])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
            **input_ids,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    print('Target: ', extract_target(math[i]['output']))
    print('Inference: ', math[i]['output'] )
    
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Answer: ', extract_answer(answer))
    print('Inference: ', answer )
    

Target:  72
Inference:  Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
Answer:  288
Inference:  Let X be the number of clips sold in April. Then, 48 + x = 96. 144 + 48 = 282. 168 + 282 = 288. Natalia sold 388 clips altogether. So the final answer is 294.
Target:  10
Inference:  Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10
Answer:  120
Inference:  Weng earns $12 x 50 = $120 for babysitting. So the final answer is 120.
Target:  5
Inference:  In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5
Answer:  90
Inference:  Betty has only one - half of the money she needs to buy the wallet, which means she has $100 / 2 = $50. Her parents gave her $15, and her grandparents twice as much as her parents, so th

KeyboardInterrupt: 

Kết quả rất tệ nếu không hề có 1 ví dụ nào cho mô hình học

**Few-shots**

Few-shots + CoT

In [91]:
acc  = 0
results = []
for i in range(3,20):
  few_shots_prompt = f"""
  You are a math assistant. Whenever you see a math expression, call the function CALC().
Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A: Natalia sold CALC(48/2)=24 clips in May.
Natalia sold CALC(48+24)=72 clips altogether in April and May.
#### 72
Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A: Weng earns $CALC(12/60)=0.2 per minute.
Working 50 minutes, she earned $CALC(0.2*50)=10,
#### 10
Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
A: In the beginning, Betty has only $CALC(100/2) = 50.
Betty's grandparents gave her $CALC(15*2)=30.
This means, Betty needs $CALC(100-50-30-15) = 5 more.
#### 5
Q: {math[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


Today, Julie was able to read 12 x 2 = 24 pages. Therefore, she is left with 120 - 24 = 96 pages. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. #### 54
He writes 3 * 2 = 6 pages a week. So he writes 6 * 52 = 384 pages in a year. The answer: 384.
There are 10 * .8 = 8 more purple flowers than yellow flowers in Mark's garden. So there are 10 + 8 = 16 purple flowers in his garden. And there are 16 * 0.25 = 4 green flowers. So in total, Mark has 10 + 16 + 4 = 44 flowers. The answer: 44.
He eats 2 large pizzas * 16 slices / large pizza = 32 slices. He also eat 2 small pizza * 8 slices = 16 slices. Altogether, he ate 32 slices + 16 slices = 48 slices. #### 48
First find the weight of the brownies: 2 pounds * 3 brownies / pound = 6 pounds. Then add the 2 pounds of jelly beans to find the final weight: 6 pounds + 2 pounds = 8 pounds. Finally, add the 8 pounds of gummy worms to the weight: 8 pounds * 2 = 16 pounds. #### 16
Alexis spent a total of 

In [92]:
print(acc/17)

0.11764705882352941


In [93]:
acc  = 0
results = []
for i in range(20):
  few_shots_prompt = f"""Q: Janet’s ducks lay 16 eggs per day.
  She eats three for breakfast every morning and bakes muffins for her friends every day with four.
  She sells the remainder at the farmers' market daily for $2 per fresh duck egg.
  How much in dollars does she make every day at the farmers' market?
 A: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
 The answer: 18.

 Q: {math[i]['input']}. Let's think step by step',
  """
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


A: Natalia sold 48 / 2 = 24 clips in May. So, she sold a total of 48 + 24 = 72 clips in April and May. The answer: 72.
A: Weng earns $12 x 50 = $120 for babysitting yesterday. The answer: 120.
A: Betty needs $100 / 2 = $50 to buy the wallet. Her grandparents gave her twice as much as her parents, so they gave her 2 * $15 = $30. So, Betty has $50 + $30 = $75. The answer: 75.
A: Today, Julie was able to read 12 x 2 = 24 pages. So, she is left with 120 - 24 = 96 pages. If she wants to read half of the remaining pages tomorrow, she should read 96 / 2 = 54 pages. The answer: 54.
A: He writes 3 * 2 = 6 pages a week. So he writes 6 * 52 = 384 pages in a year. The answer: 384.
A: There are 10 * .8 = 8 more purple flowers than yellow flowers in Mark's garden. So there are 10 + 8 = 16 purple flowers in his garden. The total number of purple and yellow flowers is 10 + 16 = 28. There are 25% as many green flowers as there are yellow and purple flowers, so there are 25 / 100 * 28 = 8 green flowers.

In [94]:
print(acc/20)

0.1


In [95]:
import re

def split_steps(text):
    text = text.replace('\n', ' ')
    math_expr = re.findall(r'[\$\d\s\+\-\*/xX=\.]+', text)

    steps = []
    for expr in math_expr:
        expr = expr.strip()
        expr = expr.replace('x', '*').replace('X', '*')  # chuẩn hóa phép nhân
        if '=' in expr and any(c.isdigit() for c in expr):
            steps.append(expr)

    return steps

In [6]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

In [7]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
task_samples = math
new_tasks = []
for task in task_samples:
    cot, answer = split_cot_and_answer(task["output"])
    task["chain_of_thought"] = cot
    task["answer"] = answer
    new_tasks.append(task)
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)

In [8]:
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return task_samples[idx]

one-shot + CoT + ART

In [9]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
            
            Now solve this task:
            Input: {query}
            Chain of Thought:"""

In [10]:
def one_shot_ART(query):
    example = find_similar_task(query)
    prompt = build_prompt(example, query)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
      **inputs,
      max_length=150,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
    print(answer)
    return answer

In [18]:
test_tasks= []
with open("/kaggle/input/math-inference/final_test_math (1).json") as f:
    test_tasks = [json.loads(line) for line in f]
for i in range(10):
    one_shot_ART(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Token indices sequence length is longer than the specified maximum sequence length for this model (768 > 512). Running this sequence through the model will result in indexing errors


She sells 16 - 3 - 4 = 9 eggs at the farmers' market. So she makes 9 * 2 = $18. The answer: 18.
18
The robe takes 2 / 2 = 1 bolt of white fiber. So it takes 2 + 1 = 3 bolts in total. The final answer: 3.
3
He increased the value of the house by 80,000 * 1.5 = $150,000. So he sold the house for 150,000 + 80,000 = $190,000. That means he made a profit of 150,000 - 190,000 = $90,000. So the final answer is 90000.
70000
He runs 60 * 3 = 180 meters a week. So the final answer is 180.
540
In the morning, Wendi gives her chickens a total of 15 + 25 = 40 cups of feed. In the final meal of the day, she needs to give her 20 chickens 3 * 20 = 60 cups of food. So the final answer is 60.
20
Every second glass costs $5 / 2 = $3. Kylar wants to buy 16 glasses, so he needs to pay $3 * 16 = $64 for them. The final answer: 64.
64
If Charleston has 4 times as many sheep as Seattle, then it has 4 * 20 = 80 sheep. Together, Charleston and Seattle have 80 + 20 = 100 sheep. Toulouse has twice as many as Char

In [19]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
            Now solve this task:
            Input: {query}
            Solution:
Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"""

In [22]:
acc = 0
for i in range(20):
    answer = one_shot_ART(test_tasks[i]['input'])
    print('Answer: ', extract_answer(answer))
    target = extract_target(test_tasks[i]['output'])
    print('Target: ', target)
    if extract_answer(answer) == target:
        acc = acc+1

She sells 16 - 3 - 4 = 9 eggs at the farmers' market. So she makes 9 * 2 = $18. The answer: 18.
Answer:  18
Target:  18
The robe takes 2 / 2 = 1 bolt of white fiber. So it takes 2 + 1 = 3 bolts in total. The final answer: 3.
Answer:  3
Target:  3
He increased the value of the house by 80,000 * 1.5 = $150,000. So he sold the house for 150,000 + 80,000 = $190,000. That means he made a profit of 150,000 - 190,000 = $90,000. So the final answer is 90000.
Answer:  90
Target:  70000
He runs 60 * 3 = 180 meters a week. So the final answer is 180.
Answer:  180
Target:  540
In the morning, Wendi gives her chickens a total of 15 + 25 = 40 cups of feed. In the final meal of the day, she needs to give her 20 chickens 3 * 20 = 60 cups of food. So the final answer is 60.
Answer:  60
Target:  20
Every second glass costs $5 / 2 = $3. Kylar wants to buy 16 glasses, so he needs to pay $3 * 16 = $64 for them. The final answer: 64.
Answer:  64
Target:  64
If Charleston has 4 times as many sheep as Seattle

In [23]:
print(acc/20)

0.2


Few-shots + CoT + ART

In [25]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}
                    
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt

In [26]:
def find_top_k_tasks(query, k=3):
    q = vectorizer.transform([query])
    scores = cosine_similarity(q, X).flatten()
    top_indices = scores.argsort()[::-1][:k]
    return [task_samples[i] for i in top_indices]

In [31]:
def few_shots_CoT_ART(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
      **inputs,
      max_length=150,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
    return answer

In [33]:
acc = 0
for i in range(20):
    answer = few_shots_CoT_ART(test_tasks[i]['input'])
    print('Inference', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', extract_target(test_tasks[i]['output']))

Inference Chain of Thought: Janet eats 3 + 4 = 7 eggs a day. She has 16 - 7 = 11 eggs left. She sells 11 * 2 = 22 eggs at the farmers' market every day. So she makes 22 / 7 = $2. Therefore, the final answer is 2.
Answer:  2
Target:  18
Inference Chain of Thought: The robe takes 2 / 2 = 1 bolt of white fiber. So it takes 2 + 1 = 3 bolts in total. The answer: 3.
Answer:  3
Target:  3
Inference Chain of Thought: He bought the house for $80,000 and put in $50,000 in repairs so he sold it for 80,000 + 50,000 = $120,000. He increased the value of the house by 150% so that means the house is worth 120,000 * 1.5 = $240,000. He made a profit of 40,000 - 80,000 = $20,000 from selling the house. So the final answer is 20000.
Answer:  20
Target:  70000
Inference Chain of Thought: He runs 3 sprints * 60 meters / sprint = 180 meters a week. So the final answer is 180.
Answer:  180
Target:  540
Inference Chain of Thought: In the morning, Wendi gives her chickens 15 + 25 = 40 cups of feed. In the fina

In [ ]:
clear_gpu()

In [ ]:
for i in range(10):
    few_shots_ART(test_tasks[i]['input'], 5)
    print(extract_target(test_tasks[i]['output']))

Làm sao để giải quyết được OOM: prompt dài ??? -> prompt compression

Few-shots + CoT(ReAct) + SELF-CONSITENCY (SC) + ART

In [34]:
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=150,
            do_sample=True,           # sampling ON
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    answers = [extract_answer(out) for out in outputs]
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [36]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}
                    
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt


In [38]:
def few_shots_CoT_SC_ART(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)

In [39]:
for i in range(20):
    few_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Chain of Thought: She eats 3 + 4 = 7 eggs. So, she has 16 - 7 = 9 left. She sells 9 * 2 = $24 at the farmers' market. So, she makes 24 / 9 = $3 at the farmers' market every day. Therefore, the final answer is 3.
Chain of Thought: She eats 3 + 4 = 7 eggs per day. She has 16 – 7 = 13 eggs left over. She sells her eggs for 13 * 2 = $24. She makes 24 * 2 = $48 per day. So the final answer is 48.
Chain of Thought: First find how many eggs Janet uses for breakfast: 16 eggs - 3 eggs - 4 eggs = 9 eggs. Then find how many eggs she sells at the farmers' market: 9 eggs / $2 / egg = 9 dollars. Then multiply her sales by the number of eggs to find her total daily earnings: $9 * $2 / egg = $18. The answer: 18.
Chain of Thought: Janet eats 3 + 4 = 7 eggs per day. Thus, she has 16 - 7 = 9 eggs left over. Janet sells her eggs for $2 x 9 = $18. Therefore, she makes $18 / 2 = $4 at the farmers' market. Therefore, the answer is 4.
Chain of Thought: Janet eats 3 + 4 = 7 eggs every day. She has 16 - 7 = 9 e

Few-shots + ToT

In [45]:
def few_shots_ToT(text):
    return f"""
   Your task: Write 1 possible reasoning path or situation.
    {text} 
    """
def followup_prompt(text):
 return f"""Here are 3 different reasoning paths:  {text}.
 You are a math assistant. Your job is to explore different reasoning paths without concluding.
 Now, expand on each path with clear explanation. The final answer is ?"""
for i in range(20):
    prompt_base = few_shots_ToT(test_tasks[i]['input'])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    prompt = prompt_base + "\n" + "\n".join(strategies) + "\n" + "Answer:"
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
            **input_ids,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    print('Target', extract_target(test_tasks[i]['output']))
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Inference', answer)
    print('Answer', extract_answer(answer))

Target 18
Inference The final answer: 24.
Answer None
Target 3
Inference Half as much white fiber as blue fiber is 2 / 2 = 1 bolt. So the robe takes 2 + 1 = 3 bolts. The answer: 3.
Answer 3
Target 70000
Inference Then subtract the cost of repairs from the sale price to find the profit: $160,000 - $80,000 = $30,000 profit.
Answer 30
Target 540
Inference He runs 60 * 3 = 180 meters in 3 sprints. So he runs 180 + 3 * 60 = 540 meters a week. The answer: 560.
Answer 540
Target 20
Inference Wendi needs to give her chickens 60 cups of feed.
Answer None
Target 64
Inference Strategy 3: Every second glass costs $5 / 2 = $3. Kylar needs to pay $3 * 16 = $64 for all the glasses. Thus, the answer is 64.
Answer 64
Target 260
Inference 300.
Answer None
Target 160
Inference First find how long Carla spends restarting the download: 20 minutes * 2 GB / minute = 40 minutes. Then subtract the restart time from the time to download the file to find the total download time: 70 minutes - 10 minutes = 60 minu

# Reasoning

In [53]:
import json
data= []
file_path = "/kaggle/input/new-infer/reasoning_entries.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

**Zero-shot**

Zero-shot + CoT

In [50]:
def zero_shot_CoT(text):
    return f"Q: {text['input']}. A: Let's think step by step and explain"

In [55]:
input = tokenizer(zero_shot_CoT(data[0]), return_tensors="pt").to('cuda')
output = model.generate(
    **input, 
    max_length=100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

To grab a page from a pdf report and attach it in email body, you can use the following method: Right click on the pdf file and select "save target as" from the context menu. Select the page you want to grab and right click on it. Select "copy" and paste it in the body of the email. So, the answer is copy and paste.


In [89]:
clear_gpu()

In [60]:
cosine = 0
for i in range(50):
    input = tokenizer(zero_shot_CoT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Answer: ', answer)
    print('Target: ', data[i]['output'])
    c = cosine_similarity_score(answer, data[i]['output'])
    print(c)
    cosine = cosine + c

Answer:  To grab a page from a pdf report and attach it in email body, you can use the following method: Right click on the pdf file and select "save target as" from the context menu. Select the page you want to grab and right click on it. Select "copy" and paste it in the body of the email. So, the answer is copy and paste.
Target:  You can extract a page from a PDF using a PDF library such as PyPDF2 or pdfminer in Python, or iText in Java. Then you can attach it to an email using an email library such as smtplib in Python, or JavaMail in Java.

Here's an example using Python and the PyPDF2 library to extract a page and attach it to an email using smtplib:
```python
import PyPDF2
import smtplib
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Extract page from PDF
pdf_file = PyPDF2.PdfFileReader(open("report.pdf", "rb"))
page = pdf_file.getPage(0)
pdf_bytes = page.extractText().encode("utf-8")

# 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.7691444158554077
Answer:  Here's what I'm talking about: If you're looking for a place where you can get away from the hustle and bustle of everyday life, you've come to the right place.
Target:  Certainly! Here are a few links to some projects that are similar to what you described:

1. "DIY Automated Cocktail Dispenser" by GreatScottLab: <https://www.instructables.com/DIY-Automated-Cocktail-Dispenser/>
2. "RFID Cocktail Machine" by zndr: <https://www.instructables.com/RFID-Cocktail-Machine/>
3. "RFID-Enabled Cocktail Dispenser" by Chirag Patel: <https://www.hackster.io/chiragpatel/rfid-enabled-cocktail-dispenser-88faeb>
4. "Automated Drink Dispenser" by MicrocontrollersAndMore: <https://www.instructables.com/Automated-Drink-Dispenser/>

These projects may not be identical to what you are envisioning, but they provide a good starting point for inspiration and reference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8320773839950562
Answer:  negative
Target:  Based on the sample reflections provided, it seems that Jared, Lisa, and David all have different sentiments regarding their workload:

* David seems to be managing his workload well and is feeling proud of his progress on his projects.
* Lisa is feeling frustrated with the lack of feedback on her work and is struggling to balance her workload with personal issues.
* Jared is feeling disconnected from the team and burnt out from his workload.

Overall, the sentiment regarding workload is mixed. While David seems to be doing well, Lisa and Jared are experiencing some challenges. It may be helpful for their manager to check in with them to see how they can provide support and improve their work experiences.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.05353083834052086


In [61]:
print(cosine/50)

0.5984805060178041


In [73]:
def zero_shot_ToT(text):
    return f"Q: {text['input']}. A: explain the main idea and how it could be implemented."
def followup_prompt(text):
 return f"Here are 3 possible situations: {text}. A: Choose the best and explain"


Zero-shot + ToT

In [77]:
prompt_base = zero_shot_ToT(data[0])
strategies = [generate_strategy(prompt_base, j) for j in range(3)]
print(strategies)
# prompt = prompt_base + "\n" +"\n".join(strategies) + "\n" + "Answer: "
input = tokenizer(followup_prompt(strategies), return_tensors="pt").to('cuda')
output = model.generate(
        **input,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
answer =tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

["You can use the program 'Acrobat Reader' to open the pdf file and then select the page you want to copy and click 'copy'. It will then save the page as a new document.", 'Use a PDF reader to open the report and select the page you want to copy. Save the copied page as a PDF.', 'Use a browser with a PDF viewer.']
["You can use the program 'Acrobat Reader' to open the pdf file and then select the page you want to copy and click 'copy'. It will then save the page as a new document."]


Zero-shot + CoT + SC

**Few-shots**

Few-shots + CoT

In [78]:
def few_shots_CoT(text):
    return f"""
Q: Explain quantum computing in simple terms,
A: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to store and process information, quantum computers use quantum bits, or qubits. Because qubits can be in multiple states at once, quantum computers are able to perform many calculations simultaneously, which can make them much faster than classical computers for certain types of problems. While quantum computers are still in the early stages of development, they have the potential to revolutionize many fields, including cryptography, medicine, and materials science.

Q: why shouldn't I trust birds?,
A: There is no reason not to trust birds in general. However, wild birds may carry diseases and it's best to avoid handling them or coming into contact with their droppings. Also, some bird species, such as pigeons, can carry diseases such as histoplasmosis and cryptococcosis. So it's better to take precautions when interacting with wild birds.

Q: I have an array of DomElements in JS, how can I find the item with the largest height?
A: You can loop through the array of DomElements, and for each element, use the `offsetHeight` property to get its height. You can then compare this height to the current maximum height found so far and update the maximum height and corresponding DomElement accordingly. Here's an example implementation:
```javascript
function findMaxHeightElement(elements) {{
  let maxHeight = -Infinity;
  let maxHeightElement = null;

  for (let i = 0; i < elements.length; i++) {{
    const elementHeight = elements[i].offsetHeight;
    if (elementHeight > maxHeight) {{
      maxHeight = elementHeight;
      maxHeightElement = elements[i];
    }}
  }}

  return maxHeightElement;
}}
Q: {text['input']}
A: Let's think step by step
"""

In [83]:
cosine = 0
for i in range(20):
    input = tokenizer(few_shots_CoT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    target = data[i]['output']
    print(answer)
    c = cosine_similarity_score(answer, target)
    print(c)
    cosine = cosine + c

I would like to grab a page from a pdf report and attach it in email body how do I go about it? So, the answer is a).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.75454181432724
Here's an example implementation: javascript function findMaxHeightElement(elements)  let maxHeight = -Infinity; let x = 0; i  elements.length; x++; if (maxHeight > elementHeight) ;  if x != x[0]: x += 1; y = x; z = y


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.724153995513916
I shared what I shared with you and explained it at a sixth grade level.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.07597992569208145


In [84]:
print(cosine/20)

0.6539904663804919


Few-shots + CoT + SC

Few-shots ToT

In [ ]:
def few_shots_ToT(text):
    return f"""
Q: Explain quantum computing in simple terms,
A: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to store and process information, quantum computers use quantum bits, or qubits. Because qubits can be in multiple states at once, quantum computers are able to perform many calculations simultaneously, which can make them much faster than classical computers for certain types of problems. While quantum computers are still in the early stages of development, they have the potential to revolutionize many fields, including cryptography, medicine, and materials science.

Q: why shouldn't I trust birds?,
A: There is no reason not to trust birds in general. However, wild birds may carry diseases and it's best to avoid handling them or coming into contact with their droppings. Also, some bird species, such as pigeons, can carry diseases such as histoplasmosis and cryptococcosis. So it's better to take precautions when interacting with wild birds.

Q: I have an array of DomElements in JS, how can I find the item with the largest height?
A: You can loop through the array of DomElements, and for each element, use the `offsetHeight` property to get its height. You can then compare this height to the current maximum height found so far and update the maximum height and corresponding DomElement accordingly. Here's an example implementation:
```javascript
function findMaxHeightElement(elements) {{
  let maxHeight = -Infinity;
  let maxHeightElement = null;

  for (let i = 0; i < elements.length; i++) {{
    const elementHeight = elements[i].offsetHeight;
    if (elementHeight > maxHeight) {{
      maxHeight = elementHeight;
      maxHeightElement = elements[i];
    }}
  }}

  return maxHeightElement;
}}
Q: {text['input']}
A:  A:Let's think of three different strategies for this question. For each strategy, explain the main idea and how it could be implemented.
"""

In [86]:
cosine = 0
for i in range(20):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    # prompt = prompt_base + "\n" +"\n".join(strategies) + "\n" + "Answer: "
    input = tokenizer(followup_prompt(strategies), return_tensors="pt").to('cuda')
    output = model.generate(
            **input,
            max_length=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            num_return_sequences=1
        )
    answer =tokenizer.decode(output[0], skip_special_tokens=True)
    target = data[i]['output']
    print(answer)
    c = cosine_similarity_score(answer, target)
    print(c)
    cosine = cosine + c

["Act as a software developer , I would like to grab a page from a pdf report and attach it in email body how do I go about it?']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.7033816576004028
The person is looking for information on how to make a cocktail dispenser.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9491288661956787
You want to know about Owners Rep


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.34278255701065063


In [87]:
print(cosine/20)

0.6443684794008732


# Finetune

In [ ]:
import json
classification = []
with open('classification_instruction_data_en.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        classification.append(json.loads(line.strip()))
qa = []
computation = []
reasoning = []

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
os.environ["WANDB_DISABLED"] = "true"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model2 = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False
)
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model2)
)
trainer.train()